In [10]:
import pandas as pd
import itertools as itt
import numbers as nb
import numpy as np
import statistics as st
import re
import math

from cleaning_utils import pivot_dataset, transform_chemical_data, create_standardise_units_func, drop_chemical_columns, order_cols

In [2]:
idaho_data = pd.read_csv("Datasets/idaho_data.csv")

/tmp/ipykernel_1201925/1784059464.py:1: DtypeWarning: Columns (13,14,17,18,19,21,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  idaho_data = pd.read_csv("Datasets/idaho_data.csv")


In [3]:
idaho_data[["SampleNumber", "CharName", "Amount", "UOM", "MinDetectLimit"]]

,SampleNumber,CharName,Amount,UOM,MinDetectLimit
0,GWQM-1993-237,Fecal Coliform,<1,col/100 ml,1.000
1,GWQM-1993-237,Alkalinity as (CaCO3),247,mg/l,257.000
2,GWQM-1993-237,Ammonia,0.03,mg/l,0.002
3,GWQM-1993-237,Arsenic,<1,ug/L,0.200
4,GWQM-1993-237,Bicarbonate,302,mg/L,NaN
...,...,...,...,...,...
885839,GWQM-2022-0218,"1H,1H, 2H, 2H-Perfluorohexane sulfonic acid (4...",<0.00200,ug/L,0.002
885840,GWQM-2022-0218,Dissolved Oxygen,4.83,mg/l,0.000
885841,GWQM-2022-0218,pH,6.8,pH,0.000
885842,GWQM-2022-0218,Specific Conductance,760.4,uS/cm,0.000


In [4]:
desired_chemical_names = ["Chloride", "Sulfate", "Hardness", "Sodium", "Potassium", "Magnesium", "Calcium",
                          "Specific Conductance", "Total Dissolved Solids", "Water Temperature", "pH"]

dataset_pivoted = pivot_dataset(
    idaho_data,
    sample_id_columns=["SampleNumber", "SampleDate"],
    per_sample_data=["Latitude", "Longitude"],
    chemical_name_column="CharName",
    values_per_chemical=["Amount", "UOM", "MinDetectLimit"],
    desired_chemical_names=desired_chemical_names,
    # drop_duplicates=True
)
dataset_pivoted

SampleNumber        SampleDate   Latitude   Longitude Calcium  \
                                                                      Amount   
0              0003ETAN655B  29/03/2000 10:20  43.847830 -112.709326  [65.6]   
1              0003ETAN655C  29/03/2000 10:20  43.847830 -112.709326     NaN   
2              0004ENRFIWDB  20/04/2000 10:05  43.651806 -112.919948   [896]   
3              0004ENRFIWDD  20/04/2000 10:05  43.651806 -112.919948     NaN   
4              0004ENRFSEWB   20/04/2000 9:40  43.653774 -112.912298   [8.4]   
...                     ...               ...        ...         ...     ...   
28690  IDEQ-2017-11-08-6159   8/11/2017 14:32  43.376230 -116.550910     NaN   
28691  IDEQ-2017-11-14-6160  14/11/2017 10:55  43.717750 -116.998090     NaN   
28692  IDEQ-2017-11-14-6161  14/11/2017 11:40  43.713400 -116.902030     NaN   
28693  IDEQ-2017-11-14-6162  14/11/2017 12:17  43.697140 -117.020360     NaN   
28694  IDEQ-2017-11-20-6163  20/11/2017 10:50  43.897710 -111.157300    [66]   

                             Chloride                         ... Sulfate  \
      MinDetectLimit     UOM   Amount MinDetectLimit     UOM  ...     UOM   
0              [0.1]  [mg/l]      NaN            NaN     NaN  ...     NaN   
1                NaN     NaN    [411]          [0.9]  [mg/l]  ...  [mg/l]   
2              [0.1]  [mg/l]      NaN            NaN     NaN  ...     NaN   
3                NaN     NaN   [9713]          [0.9]  [mg/l]  ...  [mg/l]   
4              [0.1]  [mg/l]      NaN            NaN     NaN  ...     NaN   
...              ...     ...      ...            ...     ...  ...     ...   
28690            NaN     NaN      NaN            NaN     NaN  ...     NaN   
28691            NaN     NaN      NaN            NaN     NaN  ...     NaN   
28692            NaN     NaN      NaN            NaN     NaN  ...     NaN   
28693            NaN     NaN      NaN            NaN     NaN  ...     NaN   
28694          [nan]  [mg/l]   [32.3]          [nan]  [mg/l]  ...  [mg/l]   

      Total Dissolved Solids                        Water Temperature  \
                      Amount MinDetectLimit     UOM            Amount   
0                        NaN            NaN     NaN               NaN   
1                     [1320]          [1.0]  [mg/l]               NaN   
2                        NaN            NaN     NaN               NaN   
3                    [17100]          [1.0]  [mg/l]               NaN   
4                        NaN            NaN     NaN               NaN   
...                      ...            ...     ...               ...   
28690                    NaN            NaN     NaN           [15.79]   
28691                    NaN            NaN     NaN           [18.94]   
28692                    NaN            NaN     NaN           [15.42]   
28693                    NaN            NaN     NaN           [16.51]   
28694                    NaN            NaN     NaN            [9.02]   

                                pH                       
      MinDetectLimit   UOM  Amount MinDetectLimit   UOM  
0                NaN   NaN     NaN            NaN   NaN  
1                NaN   NaN     NaN            NaN   NaN  
2                NaN   NaN     NaN            NaN   NaN  
3                NaN   NaN     NaN            NaN   NaN  
4                NaN   NaN     NaN            NaN   NaN  
...              ...   ...     ...            ...   ...  
28690          [nan]  [°C]  [7.56]          [nan]  [pH]  
28691          [nan]  [°C]  [7.14]          [nan]  [pH]  
28692          [nan]  [°C]  [6.95]          [nan]  [pH]  
28693          [nan]  [°C]  [7.42]          [nan]  [pH]  
28694          [nan]  [°C]   [6.9]          [nan]  [pH]  

[28695 rows x 37 columns]

In [5]:
def clean_units(units):
    if isinstance(units, str):
        return ("".join(units.strip().lower().split()),)
    else:
        return (np.nan,)


transform_chemical_data(
    dataset_pivoted,
    desired_chemical_names,
    clean_units,
    ["UOM"],
    ["UOM"],
    split_lists=True
)

dataset_pivoted

SampleNumber        SampleDate   Latitude   Longitude Calcium  \
                                                                      Amount   
0              0003ETAN655B  29/03/2000 10:20  43.847830 -112.709326  [65.6]   
1              0003ETAN655C  29/03/2000 10:20  43.847830 -112.709326     NaN   
2              0004ENRFIWDB  20/04/2000 10:05  43.651806 -112.919948   [896]   
3              0004ENRFIWDD  20/04/2000 10:05  43.651806 -112.919948     NaN   
4              0004ENRFSEWB   20/04/2000 9:40  43.653774 -112.912298   [8.4]   
...                     ...               ...        ...         ...     ...   
28690  IDEQ-2017-11-08-6159   8/11/2017 14:32  43.376230 -116.550910     NaN   
28691  IDEQ-2017-11-14-6160  14/11/2017 10:55  43.717750 -116.998090     NaN   
28692  IDEQ-2017-11-14-6161  14/11/2017 11:40  43.713400 -116.902030     NaN   
28693  IDEQ-2017-11-14-6162  14/11/2017 12:17  43.697140 -117.020360     NaN   
28694  IDEQ-2017-11-20-6163  20/11/2017 10:50  43.897710 -111.157300    [66]   

                             Chloride                         ... Sulfate  \
      MinDetectLimit     UOM   Amount MinDetectLimit     UOM  ...     UOM   
0              [0.1]  [mg/l]      NaN            NaN     NaN  ...     NaN   
1                NaN     NaN    [411]          [0.9]  [mg/l]  ...  [mg/l]   
2              [0.1]  [mg/l]      NaN            NaN     NaN  ...     NaN   
3                NaN     NaN   [9713]          [0.9]  [mg/l]  ...  [mg/l]   
4              [0.1]  [mg/l]      NaN            NaN     NaN  ...     NaN   
...              ...     ...      ...            ...     ...  ...     ...   
28690            NaN     NaN      NaN            NaN     NaN  ...     NaN   
28691            NaN     NaN      NaN            NaN     NaN  ...     NaN   
28692            NaN     NaN      NaN            NaN     NaN  ...     NaN   
28693            NaN     NaN      NaN            NaN     NaN  ...     NaN   
28694          [nan]  [mg/l]   [32.3]          [nan]  [mg/l]  ...  [mg/l]   

      Total Dissolved Solids                        Water Temperature  \
                      Amount MinDetectLimit     UOM            Amount   
0                        NaN            NaN     NaN               NaN   
1                     [1320]          [1.0]  [mg/l]               NaN   
2                        NaN            NaN     NaN               NaN   
3                    [17100]          [1.0]  [mg/l]               NaN   
4                        NaN            NaN     NaN               NaN   
...                      ...            ...     ...               ...   
28690                    NaN            NaN     NaN           [15.79]   
28691                    NaN            NaN     NaN           [18.94]   
28692                    NaN            NaN     NaN           [15.42]   
28693                    NaN            NaN     NaN           [16.51]   
28694                    NaN            NaN     NaN            [9.02]   

                                pH                       
      MinDetectLimit   UOM  Amount MinDetectLimit   UOM  
0                NaN   NaN     NaN            NaN   NaN  
1                NaN   NaN     NaN            NaN   NaN  
2                NaN   NaN     NaN            NaN   NaN  
3                NaN   NaN     NaN            NaN   NaN  
4                NaN   NaN     NaN            NaN   NaN  
...              ...   ...     ...            ...   ...  
28690          [nan]  [°c]  [7.56]          [nan]  [ph]  
28691          [nan]  [°c]  [7.14]          [nan]  [ph]  
28692          [nan]  [°c]  [6.95]          [nan]  [ph]  
28693          [nan]  [°c]  [7.42]          [nan]  [ph]  
28694          [nan]  [°c]   [6.9]          [nan]  [ph]  

[28695 rows x 37 columns]

In [6]:

def format_amount(erase_invalid: bool = False):

    def format_amount_func(amount, min_detection_limit, uom):
        # if amount is already number
        if isinstance(amount, nb.Number):
            return ('=', float(amount), uom)

        if isinstance(amount, str):
            # remove whitespace
            amount = ''.join(amount.split())
            amount_prefix = None

            # if amount matches prefix-numerical form
            if re.fullmatch(r'([<>=]?)[0-9]*(\.?)[0-9]+', amount):
                if re.match(r'[<>=]', amount):
                    amount_prefix = re.split(r'[<>=]', amount)
                    amount_prefix[1] = float(amount_prefix[1])
                    amount_prefix = (amount_prefix[0], amount_prefix[1], uom)
                else:
                    amount_prefix = ("=", float(amount), uom)
                return amount_prefix

            # if amount is below detection limit
            elif amount == "BDL" or amount == "ND":
                if min_detection_limit is not None:
                    return ("<", min_detection_limit, uom)
                else:
                    return ("=", 0, uom)

        # otherwise amount is invalid
        if erase_invalid:
            return (np.nan, np.nan, np.nan)
        else:
            raise ValueError("Invalid Formatting - " + str(amount))

    return format_amount_func


transform_chemical_data(
    dataset_pivoted,
    desired_chemical_names,
    format_amount(True),
    ["Amount", "MinDetectLimit", "UOM"],
    ["Prefix", "Amount", "UOM"],
    split_lists=True
)

dataset_pivoted

SampleNumber        SampleDate   Latitude   Longitude  Calcium  \
                                                                       Amount   
0              0003ETAN655B  29/03/2000 10:20  43.847830 -112.709326   [65.6]   
1              0003ETAN655C  29/03/2000 10:20  43.847830 -112.709326      NaN   
2              0004ENRFIWDB  20/04/2000 10:05  43.651806 -112.919948  [896.0]   
3              0004ENRFIWDD  20/04/2000 10:05  43.651806 -112.919948      NaN   
4              0004ENRFSEWB   20/04/2000 9:40  43.653774 -112.912298    [8.4]   
...                     ...               ...        ...         ...      ...   
28690  IDEQ-2017-11-08-6159   8/11/2017 14:32  43.376230 -116.550910      NaN   
28691  IDEQ-2017-11-14-6160  14/11/2017 10:55  43.717750 -116.998090      NaN   
28692  IDEQ-2017-11-14-6161  14/11/2017 11:40  43.713400 -116.902030      NaN   
28693  IDEQ-2017-11-14-6162  14/11/2017 12:17  43.697140 -117.020360      NaN   
28694  IDEQ-2017-11-20-6163  20/11/2017 10:50  43.897710 -111.157300   [66.0]   

                              Chloride                         ... Sulfate  \
      MinDetectLimit     UOM    Amount MinDetectLimit     UOM  ...  Prefix   
0              [0.1]  [mg/l]       NaN            NaN     NaN  ...       =   
1                NaN     NaN   [411.0]          [0.9]  [mg/l]  ...     [=]   
2              [0.1]  [mg/l]       NaN            NaN     NaN  ...       =   
3                NaN     NaN  [9713.0]          [0.9]  [mg/l]  ...     [=]   
4              [0.1]  [mg/l]       NaN            NaN     NaN  ...       =   
...              ...     ...       ...            ...     ...  ...     ...   
28690            NaN     NaN       NaN            NaN     NaN  ...       =   
28691            NaN     NaN       NaN            NaN     NaN  ...       =   
28692            NaN     NaN       NaN            NaN     NaN  ...       =   
28693            NaN     NaN       NaN            NaN     NaN  ...       =   
28694          [nan]  [mg/l]    [32.3]          [nan]  [mg/l]  ...     [=]   

      Hardness Sodium Potassium Magnesium Calcium Specific Conductance  \
        Prefix Prefix    Prefix    Prefix  Prefix               Prefix   
0            =    [=]       [=]       [=]     [=]                    =   
1            =      =         =         =       =                    =   
2            =    [=]       [=]       [=]     [=]                    =   
3            =      =         =         =       =                    =   
4            =    [=]       [=]       [=]     [=]                    =   
...        ...    ...       ...       ...     ...                  ...   
28690        =      =         =         =       =                  [=]   
28691        =      =         =         =       =                  [=]   
28692        =      =         =         =       =                  [=]   
28693        =      =         =         =       =                  [=]   
28694        =    [=]       [=]       [=]     [=]                  [=]   

      Total Dissolved Solids Water Temperature     pH  
                      Prefix            Prefix Prefix  
0                          =                 =      =  
1                        [=]                 =      =  
2                          =                 =      =  
3                        [=]                 =      =  
4                          =                 =      =  
...                      ...               ...    ...  
28690                      =               [=]    [=]  
28691                      =               [=]    [=]  
28692                      =               [=]    [=]  
28693                      =               [=]    [=]  
28694                      =               [=]    [=]  

[28695 rows x 48 columns]

In [7]:
convert_to_standard = dict()

# mass/volume concentration g/L
convert_to_standard['g/l'] = lambda x: x
convert_to_standard['mg/l'] = lambda x: x/1e3
convert_to_standard['ug/l'] = lambda x: x/1e6
convert_to_standard['µg/l'] = lambda x: x/1e6
convert_to_standard['ng/l'] = lambda x: x/1e9

# pH
convert_to_standard['ph'] = lambda x: x

# temperature °c
convert_to_standard['°c'] = lambda x: x
convert_to_standard['°f'] = lambda x: 5*(x-32)/9

# conductivity us/cm
convert_to_standard['us/cm'] = lambda x: x
convert_to_standard['µs/cm'] = lambda x: x


# apply conversion to both the amount and the mindetection limit
transform_chemical_data(
    dataset_pivoted,
    desired_chemical_names,
    create_standardise_units_func(convert_to_standard),
    ["Amount", "UOM"],
    ["Amount"],
    split_lists=True
)

# apply conversion
transform_chemical_data(
    dataset_pivoted,
    desired_chemical_names,
    create_standardise_units_func(convert_to_standard),
    ["MinDetectLimit", "UOM"],
    ["MinDetectLimit"],
    split_lists=True
)

dataset_pivoted

SampleNumber        SampleDate   Latitude   Longitude  \
                                                                       
0              0003ETAN655B  29/03/2000 10:20  43.847830 -112.709326   
1              0003ETAN655C  29/03/2000 10:20  43.847830 -112.709326   
2              0004ENRFIWDB  20/04/2000 10:05  43.651806 -112.919948   
3              0004ENRFIWDD  20/04/2000 10:05  43.651806 -112.919948   
4              0004ENRFSEWB   20/04/2000 9:40  43.653774 -112.912298   
...                     ...               ...        ...         ...   
28690  IDEQ-2017-11-08-6159   8/11/2017 14:32  43.376230 -116.550910   
28691  IDEQ-2017-11-14-6160  14/11/2017 10:55  43.717750 -116.998090   
28692  IDEQ-2017-11-14-6161  14/11/2017 11:40  43.713400 -116.902030   
28693  IDEQ-2017-11-14-6162  14/11/2017 12:17  43.697140 -117.020360   
28694  IDEQ-2017-11-20-6163  20/11/2017 10:50  43.897710 -111.157300   

                      Calcium                                       Chloride  \
                       Amount MinDetectLimit     UOM                  Amount   
0       [0.06559999999999999]       [0.0001]  [mg/l]                     NaN   
1                         NaN            NaN     NaN                 [0.411]   
2                     [0.896]       [0.0001]  [mg/l]                     NaN   
3                         NaN            NaN     NaN                 [9.713]   
4      [0.008400000000000001]       [0.0001]  [mg/l]                     NaN   
...                       ...            ...     ...                     ...   
28690                     NaN            NaN     NaN                     NaN   
28691                     NaN            NaN     NaN                     NaN   
28692                     NaN            NaN     NaN                     NaN   
28693                     NaN            NaN     NaN                     NaN   
28694                 [0.066]          [nan]  [mg/l]  [0.032299999999999995]   

                              ... Sulfate Hardness Sodium Potassium Magnesium  \
      MinDetectLimit     UOM  ...  Prefix   Prefix Prefix    Prefix    Prefix   
0                NaN     NaN  ...       =        =    [=]       [=]       [=]   
1           [0.0009]  [mg/l]  ...     [=]        =      =         =         =   
2                NaN     NaN  ...       =        =    [=]       [=]       [=]   
3           [0.0009]  [mg/l]  ...     [=]        =      =         =         =   
4                NaN     NaN  ...       =        =    [=]       [=]       [=]   
...              ...     ...  ...     ...      ...    ...       ...       ...   
28690            NaN     NaN  ...       =        =      =         =         =   
28691            NaN     NaN  ...       =        =      =         =         =   
28692            NaN     NaN  ...       =        =      =         =         =   
28693            NaN     NaN  ...       =        =      =         =         =   
28694          [nan]  [mg/l]  ...     [=]        =    [=]       [=]       [=]   

      Calcium Specific Conductance Total Dissolved Solids Water Temperature  \
       Prefix               Prefix                 Prefix            Prefix   
0         [=]                    =                      =                 =   
1           =                    =                    [=]                 =   
2         [=]                    =                      =                 =   
3           =                    =                    [=]                 =   
4         [=]                    =                      =                 =   
...       ...                  ...                    ...               ...   
28690       =                  [=]                      =               [=]   
28691       =                  [=]                      =               [=]   
28692       =                  [=]                      =               [=]   
28693       =                  [=]                      =               [=]   
28694     [=]                  [=] 

In [8]:
drop_chemical_columns(
    dataset_pivoted,
    desired_chemical_names,
    ["UOM", "MinDetectLimit"]
)

dataset_pivoted = order_cols(dataset_pivoted, desired_chemical_names)

In [11]:
# turn list of measurements into one aggregated measurement

def agg_measurement(amount, prefix):
    if not isinstance(amount, list):
        if math.isnan(amount):
            return (np.nan, np.nan)
        elif isinstance(amount, nb.Number):
            if isinstance(prefix, list):
                raise TypeError(prefix)
            return (amount, prefix)
        else:
            raise TypeError(amount)

    if set(prefix) == set(["<"]):
        return (min(amount), "<")
    else:
        return (st.mean(amount), "=")


transform_chemical_data(
    dataset_pivoted,
    desired_chemical_names,
    agg_measurement,
    ["Amount", "Prefix"],
    ["Amount", "Prefix"]
)

dataset_pivoted

SampleDate   Longitude   Latitude          SampleNumber Calcium  \
                                                                      Amount   
0      29/03/2000 10:20 -112.709326  43.847830          0003ETAN655B  0.0656   
1      29/03/2000 10:20 -112.709326  43.847830          0003ETAN655C     NaN   
2      20/04/2000 10:05 -112.919948  43.651806          0004ENRFIWDB  0.8960   
3      20/04/2000 10:05 -112.919948  43.651806          0004ENRFIWDD     NaN   
4       20/04/2000 9:40 -112.912298  43.653774          0004ENRFSEWB  0.0084   
...                 ...         ...        ...                   ...     ...   
28690   8/11/2017 14:32 -116.550910  43.376230  IDEQ-2017-11-08-6159     NaN   
28691  14/11/2017 10:55 -116.998090  43.717750  IDEQ-2017-11-14-6160     NaN   
28692  14/11/2017 11:40 -116.902030  43.713400  IDEQ-2017-11-14-6161     NaN   
28693  14/11/2017 12:17 -117.020360  43.697140  IDEQ-2017-11-14-6162     NaN   
28694  20/11/2017 10:50 -111.157300  43.897710  IDEQ-2017-11-20-6163  0.0660   

             Chloride        Hardness         ... Specific Conductance         \
      Prefix   Amount Prefix   Amount Prefix  ...               Amount Prefix   
0          =      NaN    NaN      NaN    NaN  ...                  NaN    NaN   
1        NaN   0.4110      =      NaN    NaN  ...                  NaN    NaN   
2          =      NaN    NaN      NaN    NaN  ...                  NaN    NaN   
3        NaN   9.7130      =      NaN    NaN  ...                  NaN    NaN   
4          =      NaN    NaN      NaN    NaN  ...                  NaN    NaN   
...      ...      ...    ...      ...    ...  ...                  ...    ...   
28690    NaN      NaN    NaN      NaN    NaN  ...                817.0      =   
28691    NaN      NaN    NaN      NaN    NaN  ...                494.0      =   
28692    NaN      NaN    NaN      NaN    NaN  ...               1270.0      =   
28693    NaN      NaN    NaN      NaN    NaN  ...                415.0      =   
28694      =   0.0323      =      NaN    NaN  ...                699.0      =   

      Sulfate        Total Dissolved Solids        Water Temperature         \
       Amount Prefix                 Amount Prefix            Amount Prefix   
0         NaN    NaN                    NaN    NaN               NaN    NaN   
1      0.3020      =                   1.32      =               NaN    NaN   
2         NaN    NaN                    NaN    NaN               NaN    NaN   
3      0.0246      =                  17.10      =               NaN    NaN   
4         NaN    NaN                    NaN    NaN               NaN    NaN   
...       ...    ...                    ...    ...               ...    ...   
28690     NaN    NaN                    NaN    NaN             15.79      =   
28691     NaN    NaN                    NaN    NaN             18.94      =   
28692     NaN    NaN                    NaN    NaN             15.42      =   
28693     NaN    NaN                    NaN    NaN             16.51      =   
28694  0.0324      =                    NaN    NaN              9.02      =   

          pH         
      Amount Prefix  
0        NaN    NaN  
1        NaN    NaN  
2        NaN    NaN  
3        NaN    NaN  
4        NaN    NaN  
...      ...    ...  
28690   7.56      =  
28691   7.14      =  
28692   6.95      =  
28693   7.42      =  
28694   6.90      =  

[28695 rows x 26 columns]

In [38]:

def count_nas(row):
    return row.isna().sum()


na_threshold = 5
amounts_index = list(itt.product(desired_chemical_names, ["Amount"]))
mask = dataset_pivoted[amounts_index].apply(count_nas, axis=1)
mask = mask[mask > na_threshold].index

dataset_pivoted = dataset_pivoted.loc[mask]
dataset_pivoted = dataset_pivoted.reset_index()
dataset_pivoted = dataset_pivoted.drop(['index'], axis=1, level=0)

dataset_pivoted

level_0        SampleDate   Longitude   Latitude          SampleNumber  \
                                                                               
0           0  29/03/2000 10:20 -112.709326  43.847830          0003ETAN655B   
1           1  29/03/2000 10:20 -112.709326  43.847830          0003ETAN655C   
2           2  20/04/2000 10:05 -112.919948  43.651806          0004ENRFIWDB   
3           3  20/04/2000 10:05 -112.919948  43.651806          0004ENRFIWDD   
4           4   20/04/2000 9:40 -112.912298  43.653774          0004ENRFSEWB   
...       ...               ...         ...        ...                   ...   
16470   16470   8/11/2017 11:41 -116.989560  43.704780  IDEQ-2017-11-08-6157   
16471   16471   8/11/2017 14:32 -116.550910  43.376230  IDEQ-2017-11-08-6159   
16472   16472  14/11/2017 10:55 -116.998090  43.717750  IDEQ-2017-11-14-6160   
16473   16473  14/11/2017 11:40 -116.902030  43.713400  IDEQ-2017-11-14-6161   
16474   16474  14/11/2017 12:17 -117.020360  43.697140  IDEQ-2017-11-14-6162   

      Calcium        Chloride        Hardness  ... Specific Conductance  \
       Amount Prefix   Amount Prefix   Amount  ...               Amount   
0      0.0656      =      NaN    NaN      NaN  ...                  NaN   
1         NaN    NaN    0.411      =      NaN  ...                  NaN   
2      0.8960      =      NaN    NaN      NaN  ...                  NaN   
3         NaN    NaN    9.713      =      NaN  ...                  NaN   
4      0.0084      =      NaN    NaN      NaN  ...                  NaN   
...       ...    ...      ...    ...      ...  ...                  ...   
16470     NaN    NaN      NaN    NaN      NaN  ...               1300.0   
16471     NaN    NaN      NaN    NaN      NaN  ...                817.0   
16472     NaN    NaN      NaN    NaN      NaN  ...                494.0   
16473     NaN    NaN      NaN    NaN      NaN  ...               1270.0   
16474     NaN    NaN      NaN    NaN      NaN  ...                415.0   

             Sulfate        Total Dissolved Solids        Water Temperature  \
      Prefix  Amount Prefix                 Amount Prefix            Amount   
0        NaN     NaN    NaN                    NaN    NaN               NaN   
1        NaN  0.3020      =                   1.32      =               NaN   
2        NaN     NaN    NaN                    NaN    NaN               NaN   
3        NaN  0.0246      =                  17.10      =               NaN   
4        NaN     NaN    NaN                    NaN    NaN               NaN   
...      ...     ...    ...                    ...    ...               ...   
16470      =     NaN    NaN                    NaN    NaN             14.55   
16471      =     NaN    NaN                    NaN    NaN             15.79   
16472      =     NaN    NaN                    NaN    NaN             18.94   
16473      =     NaN    NaN                    NaN    NaN             15.42   
16474      =     NaN    NaN                    NaN    NaN             16.51   

                 pH         
      Prefix Amount Prefix  
0        NaN    NaN    NaN  
1        NaN    NaN    NaN  
2        NaN    NaN    NaN  
3        NaN    NaN    NaN  
4        NaN    NaN    NaN  
...      ...    ...    ...  
16470      =   7.25      =  
16471      =   7.56      =  
16472      =   7.14      =  
16473      =   6.95      =  
16474      =   7.42      =  

[16475 rows x 27 columns]

In [ ]:
# calculate averages for each chemical
# for now I naively take the mean regardless of < or =.
amount_avgs = dict()
missing_chemicals = []

for chemical_name in desired_chemical_names:
    amount_avgs[chemical_name] = dataset_pivoted[chemical_name,
                                                 "Amount"].mean()
    if math.isnan(amount_avgs[chemical_name]):
        missing_chemicals.append(chemical_name)

# remove chemicals with all NaNs
desired_chemical_names = list(
    set(desired_chemical_names).difference(set(missing_chemicals)))
dataset_pivoted = dataset_pivoted.drop(missing_chemicals, axis=1)


# fill NaN values with averages
transform_chemical_data(
    dataset_pivoted, desired_chemical_names,
    lambda a, p: (amount_avgs[chemical_name],
                  '=') if math.isnan(a) else (a, p),
    ["Amount", "Prefix"],
    ["Amount", "Prefix"]
)

dataset_pivoted

Latitude        SampleDate          SampleNumber   Longitude  \
                                                                       
0      43.847830  29/03/2000 10:20          0003ETAN655B -112.709326   
1      43.847830  29/03/2000 10:20          0003ETAN655C -112.709326   
2      43.651806  20/04/2000 10:05          0004ENRFIWDB -112.919948   
3      43.651806  20/04/2000 10:05          0004ENRFIWDD -112.919948   
4      43.653774   20/04/2000 9:40          0004ENRFSEWB -112.912298   
...          ...               ...                   ...         ...   
16470  43.704780   8/11/2017 11:41  IDEQ-2017-11-08-6157 -116.989560   
16471  43.376230   8/11/2017 14:32  IDEQ-2017-11-08-6159 -116.550910   
16472  43.717750  14/11/2017 10:55  IDEQ-2017-11-14-6160 -116.998090   
16473  43.713400  14/11/2017 11:40  IDEQ-2017-11-14-6161 -116.902030   
16474  43.697140  14/11/2017 12:17  IDEQ-2017-11-14-6162 -117.020360   

        Calcium         Chloride        Magnesium         ...  \
         Amount Prefix    Amount Prefix    Amount Prefix  ...   
0      0.065600      =  8.344642      =  0.019800      =  ...   
1      8.344642      =  0.411000      =  8.344642      =  ...   
2      0.896000      =  8.344642      =  0.289000      =  ...   
3      8.344642      =  9.713000      =  8.344642      =  ...   
4      0.008400      =  8.344642      =  0.002000      =  ...   
...         ...    ...       ...    ...       ...    ...  ...   
16470  8.344642      =  8.344642      =  8.344642      =  ...   
16471  8.344642      =  8.344642      =  8.344642      =  ...   
16472  8.344642      =  8.344642      =  8.344642      =  ...   
16473  8.344642      =  8.344642      =  8.344642      =  ...   
16474  8.344642      =  8.344642      =  8.344642      =  ...   

      Specific Conductance          Sulfate        Total Dissolved Solids  \
                    Amount Prefix    Amount Prefix                 Amount   
0                 8.344642      =  8.344642      =               8.344642   
1                 8.344642      =  0.302000      =               1.320000   
2                 8.344642      =  8.344642      =               8.344642   
3                 8.344642      =  0.024600      =              17.100000   
4                 8.344642      =  8.344642      =               8.344642   
...                    ...    ...       ...    ...                    ...   
16470          1300.000000      =  8.344642      =               8.344642   
16471           817.000000      =  8.344642      =               8.344642   
16472           494.000000      =  8.344642      =               8.344642   
16473          1270.000000      =  8.344642      =               8.344642   
16474           415.000000      =  8.344642      =               8.344642   

             Water Temperature               pH         
      Prefix            Amount Prefix    Amount Prefix  
0          =          8.344642      =  8.344642      =  
1          =          8.344642      =  8.344642      =  
2          =          8.344642      =  8.344642      =  
3          =          8.344642      =  8.344642      =  
4          =          8.344642      =  8.344642      =  
...      ...               ...    ...       ...    ...  
16470      =         14.550000      =  7.250000      =  
16471      =         15.790000      =  7.560000      =  
16472      =         18.940000      =  7.140000      =  
16473      =         15.420000      =  6.950000      =  
16474      =         16.510000      =  7.420000      =  

[16475 rows x 24 columns]

In [ ]:
drop_chemical_columns(
    dataset_pivoted,
    desired_chemical_names,
    ["Prefix"]
)
dataset_pivoted

,Latitude,SampleDate,SampleNumber,Longitude,Calcium,Chloride,Magnesium,Potassium,Sodium,Specific Conductance,Sulfate,Total Dissolved Solids,Water Temperature,pH
,,,,,Amount,Amount,Amount,Amount,Amount,Amount,Amount,Amount,Amount,Amount
0,43.847830,29/03/2000 10:20,0003ETAN655B,-112.709326,0.065600,8.344642,0.019800,0.009900,0.410000,8.344642,8.344642,8.344642,8.344642,8.344642
1,43.847830,29/03/2000 10:20,0003ETAN655C,-112.709326,8.344642,0.411000,8.344642,8.344642,8.344642,8.344642,0.302000,1.320000,8.344642,8.344642
2,43.651806,20/04/2000 10:05,0004ENRFIWDB,-112.919948,0.896000,8.344642,0.289000,0.029000,5.500000,8.344642,8.344642,8.344642,8.344642,8.344642
3,43.651806,20/04/2000 10:05,0004ENRFIWDD,-112.919948,8.344642,9.713000,8.344642,8.344642,8.344642,8.344642,0.024600,17.100000,8.344642,8.344642
4,43.653774,20/04/2000 9:40,0004ENRFSEWB,-112.912298,0.008400,8.344642,0.002000,0.014000,0.230000,8.344642,8.344642,8.344642,8.344642,8.344642
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16470,43.704780,8/11/2017 11:41,IDEQ-2017-11-08-6157,-116.989560,8.344642,8.344642,8.344642,8.344642,8.344642,1300.000000,8.344642,8.344642,14.550000,7.250000
16471,43.376230,8/11/2017 14:32,IDEQ-2017-11-08-6159,-116.550910,8.344642,8.344642,8.344642,8.344642,8.344642,817.000000,8.344642,8.344642,15.790000,7.560000
16472,43.717750,14/11/2017 10:55,IDEQ-2017-11-14-6160,-116.998090,8.344642,8.344642,8.344642,8.344642,8.344642,494.000000,8.344642,8.344642,18.940000,7.140000


In [ ]:
dataset_pivoted.columns = dataset_pivoted.columns.droplevel(1)
dataset_pivoted

,Latitude,SampleDate,SampleNumber,Longitude,Calcium,Chloride,Magnesium,Potassium,Sodium,Specific Conductance,Sulfate,Total Dissolved Solids,Water Temperature,pH
0,43.847830,29/03/2000 10:20,0003ETAN655B,-112.709326,0.065600,8.344642,0.019800,0.009900,0.410000,8.344642,8.344642,8.344642,8.344642,8.344642
1,43.847830,29/03/2000 10:20,0003ETAN655C,-112.709326,8.344642,0.411000,8.344642,8.344642,8.344642,8.344642,0.302000,1.320000,8.344642,8.344642
2,43.651806,20/04/2000 10:05,0004ENRFIWDB,-112.919948,0.896000,8.344642,0.289000,0.029000,5.500000,8.344642,8.344642,8.344642,8.344642,8.344642
3,43.651806,20/04/2000 10:05,0004ENRFIWDD,-112.919948,8.344642,9.713000,8.344642,8.344642,8.344642,8.344642,0.024600,17.100000,8.344642,8.344642
4,43.653774,20/04/2000 9:40,0004ENRFSEWB,-112.912298,0.008400,8.344642,0.002000,0.014000,0.230000,8.344642,8.344642,8.344642,8.344642,8.344642
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16470,43.704780,8/11/2017 11:41,IDEQ-2017-11-08-6157,-116.989560,8.344642,8.344642,8.344642,8.344642,8.344642,1300.000000,8.344642,8.344642,14.550000,7.250000
16471,43.376230,8/11/2017 14:32,IDEQ-2017-11-08-6159,-116.550910,8.344642,8.344642,8.344642,8.344642,8.344642,817.000000,8.344642,8.344642,15.790000,7.560000
16472,43.717750,14/11/2017 10:55,IDEQ-2017-11-14-6160,-116.998090,8.344642,8.344642,8.344642,8.344642,8.344642,494.000000,8.344642,8.344642,18.940000,7.140000
16473,43.713400,14/11/2017 11:40,IDEQ-2017-11-14-6161,-116.902030,8.344642,8.344642,8.344642,8.344642,8.344642,1270.000000,8.344642,8.344642,15.420000,6.950000
